In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor,RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score, confusion_matrix,roc_auc_score,roc_curve,auc,RocCurveDisplay, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score
import sketch
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler


import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import KNNImputer

In [ ]:
data2014 = pd.read_csv(r'C:\Учеба\Диплом\2014_Financial_Data.csv')

In [ ]:
data2015 = pd.read_csv(r'C:\Учеба\Диплом\2015_Financial_Data.csv')

In [ ]:
data2016 = pd.read_csv(r'C:\Учеба\Диплом\2016_Financial_Data.csv')

In [ ]:
data2017 = pd.read_csv(r'C:\Учеба\Диплом\2017_Financial_Data.csv')

In [ ]:
data2018 = pd.read_csv(r'C:\Учеба\Диплом\2018_Financial_Data.csv')

In [ ]:
data2014.rename(columns={"2015 PRICE VAR [%]": "Price_Var"},inplace=True)
data2015.rename(columns={"2016 PRICE VAR [%]": "Price_Var"},inplace=True)
data2016.rename(columns={"2017 PRICE VAR [%]": "Price_Var"},inplace=True)
data2017.rename(columns={"2018 PRICE VAR [%]": "Price_Var"},inplace=True)
data2018.rename(columns={"2019 PRICE VAR [%]": "Price_Var"},inplace=True)

In [ ]:
stocks = list(set(data2014['Unnamed: 0']) & set(data2015['Unnamed: 0']) & set(data2016['Unnamed: 0']) & set(data2017['Unnamed: 0']) & set(data2018['Unnamed: 0']))

In [ ]:
data2014_copy = data2014[data2014["Unnamed: 0"].isin(stocks)]
data2015_copy = data2015[data2015["Unnamed: 0"].isin(stocks)]
data2016_copy = data2016[data2016["Unnamed: 0"].isin(stocks)]
data2017_copy = data2017[data2017["Unnamed: 0"].isin(stocks)]
data2018_copy = data2018[data2018["Unnamed: 0"].isin(stocks)]

In [ ]:
for i in [data2014_copy,data2015_copy,data2016_copy,data2017_copy,data2018_copy]:
    print(len(i.columns[i.isna().sum() < 1000]))

In [ ]:
rest_columns = data2014.columns[data2014.isna().sum() < 1000]

In [ ]:
data2014_copy2 = data2014_copy[rest_columns]
data2015_copy2 = data2015_copy[rest_columns]
data2016_copy2 = data2016_copy[rest_columns]
data2017_copy2 = data2017_copy[rest_columns]
data2018_copy2 = data2018_copy[rest_columns]

In [ ]:
data2014_copy2.sort_values(by='Unnamed: 0',inplace=True,ignore_index=True)
data2015_copy2.sort_values(by='Unnamed: 0',inplace=True,ignore_index=True)
data2016_copy2.sort_values(by='Unnamed: 0',inplace=True,ignore_index=True)
data2017_copy2.sort_values(by='Unnamed: 0',inplace=True,ignore_index=True)
data2018_copy2.sort_values(by='Unnamed: 0',inplace=True,ignore_index=True)

In [ ]:
data2014_copy2.drop(columns=["Unnamed: 0", "Price_Var"],inplace=True)
data2015_copy2.drop(columns=["Unnamed: 0", "Price_Var"],inplace=True)
data2016_copy2.drop(columns=["Unnamed: 0", "Price_Var"],inplace=True)
data2017_copy2.drop(columns=["Unnamed: 0", "Price_Var"],inplace=True)
data2018_copy2.drop(columns=["Unnamed: 0", "Price_Var"],inplace=True)

In [ ]:
data2014_copy2=pd.concat([data2014_copy2, pd.get_dummies(data2014_copy2["Sector"])],axis=1)
data2015_copy2=pd.concat([data2015_copy2, pd.get_dummies(data2015_copy2["Sector"])],axis=1)
data2016_copy2=pd.concat([data2016_copy2, pd.get_dummies(data2016_copy2["Sector"])],axis=1)
data2017_copy2=pd.concat([data2017_copy2, pd.get_dummies(data2017_copy2["Sector"])],axis=1)
data2018_copy2=pd.concat([data2018_copy2, pd.get_dummies(data2018_copy2["Sector"])],axis=1)

In [ ]:
data2014_copy2.drop(columns=["Sector"],inplace=True)
data2015_copy2.drop(columns=["Sector"],inplace=True)
data2016_copy2.drop(columns=["Sector"],inplace=True)
data2017_copy2.drop(columns=["Sector"],inplace=True)
data2018_copy2.drop(columns=["Sector"],inplace=True)

In [ ]:
data2014_copy2.fillna(data2014_copy2.median(),inplace=True)
data2015_copy2.fillna(data2015_copy2.median(),inplace=True)
data2016_copy2.fillna(data2016_copy2.median(),inplace=True)
data2017_copy2.fillna(data2017_copy2.median(),inplace=True)
data2018_copy2.fillna(data2018_copy2.median(),inplace=True)

In [ ]:
def get_not_cor_cols(X, threshold):
  
  # Calculate the correlation matrix
  corr_matrix = X.corr().abs()

  # Select upper triangle of correlation matrix
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

  # Find features with correlation greater than threshold
  to_keep = [column for column in upper.columns if any(upper[column] < threshold)]

  # Drop correlated features
  # X_reduced = X.drop(to_keep, axis=1)

  return to_keep

In [ ]:
X = data2014_copy2.drop('Class', axis=1)
y = data2018_copy2['Class']

not_cor_cols = get_not_cor_cols(X, threshold=0.999)

In [ ]:
data2014_copy3 = data2014_copy2[[*not_cor_cols,"Class"]]
data2015_copy3 = data2015_copy2[[*not_cor_cols,"Class"]]
data2016_copy3 = data2016_copy2[[*not_cor_cols,"Class"]]
data2017_copy3 = data2017_copy2[[*not_cor_cols,"Class"]]
data2018_copy3 = data2018_copy2[[*not_cor_cols,"Class"]]

In [ ]:
data2014_copy4 = data2014_copy3.drop('Class', axis=1)
data2015_copy4 = data2015_copy3.drop('Class', axis=1)
data2016_copy4 = data2016_copy3.drop('Class', axis=1)
data2017_copy4 = data2017_copy3.drop('Class', axis=1)
data2018_copy4 = data2018_copy3.drop('Class', axis=1)

In [ ]:
scaler = RobustScaler()

X_2014_scaled = scaler.fit_transform(data2014_copy4)
X_2015_scaled = scaler.fit_transform(data2015_copy4)
X_2016_scaled = scaler.fit_transform(data2016_copy4)
X_2017_scaled = scaler.fit_transform(data2017_copy4)
X_2018_scaled = scaler.fit_transform(data2018_copy4)

In [ ]:
scs = np.asarray([2.72**1,2.72**2,2.72**3,2.72**4,2.72**5])/sum([2.72**1,2.72**2,2.72**3,2.72**4,2.72**5])

In [ ]:
scs = [0.05,0.1,0.15,0.2,0.5]

In [ ]:
final_df = scs[0]*X_2014_scaled+scs[1]*X_2015_scaled+scs[2]*X_2016_scaled+scs[3]*X_2017_scaled+scs[4]*X_2018_scaled

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(final_df, y)


# 3. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  # Adjust test_size and random_state as needed

# 4. Train GradientBoostingClassifier



best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}

model = GradientBoostingClassifier(**best_params)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# 2. Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Adjust 'average' parameter as needed
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 3. Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

## Сделаем предикт на 2018 год по модели 2014

In [ ]:
cols2018 = X_new.columns

In [ ]:
X = data2018.drop('Class', axis=1)
y = data2018['Class']

# X_new = remove_correlated_features(X, threshold=0.999)
X_new = X[cols2018]

In [ ]:
# 2. Standardize features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_new)

In [ ]:
y_pred = model.predict(X_scaled)

# 2. Calculate evaluation metrics
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred, average='weighted')  # Adjust 'average' parameter as needed
recall = recall_score(y, y_pred, average='weighted')
f1 = f1_score(y, y_pred, average='weighted')

# 3. Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
cm = confusion_matrix(y, y_pred)
print("Confusion Matrix:\n", cm)

## Объединим все года

In [ ]:
data2014["Year"]=2014
data2015["Year"]=2015
data2016["Year"]=2016
data2017["Year"]=2017
data2018["Year"]=2018

In [ ]:
data_all=pd.concat([data2014,data2015,data2016,data2017,data2018])

In [ ]:
X = data_all.drop('Class', axis=1)
y = data_all['Class']

X_new = remove_correlated_features(X, threshold=0.999)

In [ ]:
# 2. Standardize features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_new)

In [ ]:
# 2. Standardize features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_new)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

# 3. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  # Adjust test_size and random_state as needed

# 4. Train GradientBoostingClassifier

best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}

model = GradientBoostingClassifier(**best_params)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# 2. Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Adjust 'average' parameter as needed
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 3. Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

In [ ]:
a=model.feature_importances_

## Снова попробуем почистить данные

In [ ]:
per_share_columns = list(filter(lambda x: "per Share" in x, list(data2014.columns)))

In [ ]:
data2014.drop(columns=per_share_columns,inplace=True)

In [ ]:
X = data2014.drop('Class', axis=1)
y = data2014['Class']

X_new = remove_correlated_features(X, threshold=0.999)

In [ ]:
# 2. Standardize features
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X_new)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

# 3. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)  # Adjust test_size and random_state as needed

# 4. Train GradientBoostingClassifier

best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}

model = GradientBoostingClassifier(**best_params)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# 2. Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Adjust 'average' parameter as needed
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 3. Print the metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

In [ ]:
margin_columns = list(filter(lambda x: "Margin" in x, list(data2014.columns)))

In [ ]:
data2014.drop(columns=margin_columns,inplace=True)

In [ ]:
data2014.drop(columns=['Price to Sales Ratio', 'Payout Ratio', 'priceToBookRatio', 'priceToSalesRatio'], inplace=True)

## Новые колонки со старой функцией

In [ ]:
def get_price_var_col(year):
    return f"{year+1} PRICE VAR [%]"

In [ ]:
def preprocess_cf2_clean_rb_log(df: pd.DataFrame,price_var_col) -> pd.DataFrame:
    clean_columns = list(filter(lambda x: "per Share" not in x and "Margin" not in x and "growth" not in x.lower(), list(df.columns)))

    df_clean = df[clean_columns]
    
    sector_dict = {}
    tmp = df_clean[['Sector', 'Class']]
    for i in tmp['Sector'].drop_duplicates():
        a=tmp[tmp['Sector']==i].value_counts().sort_index().values
        sector_dict[i] = a[1]/sum(a)
    df_copy = df_clean
    df_copy['Sector'] = df_copy['Sector'].replace(sector_dict)
    df_copy.drop(columns=['Unnamed: 0', price_var_col, 'Price to Sales Ratio', 'Payout Ratio', 'priceToBookRatio', 'priceToSalesRatio'],axis=1,inplace=True)
    float_columns=list(df_copy.select_dtypes(include=['float64']).columns)
    float_columns.remove('Sector')


    df_for_loging = df_copy.drop(columns=['Class'])
    
    df_copy_pos = df_for_loging-df_for_loging.min()
    
    df_copy_pos_logged = np.log1p(df_copy_pos)

    df_copy_pos_logged['Class'] = df_copy['Class']
    

    scaler = RobustScaler()
    df2 = df_copy_pos_logged
    df2[float_columns] = scaler.fit_transform(df2[float_columns])
    
    lower = df2[float_columns].quantile(0.01)
    greater = df2[float_columns].quantile(0.99)
    df2[float_columns] = (df2[float_columns])[(df2[float_columns] >= lower) & (df2[float_columns] <= greater)]

    df2.fillna(df2.median(),inplace=True)

    return df2

In [ ]:
data2014_pp_rb = preprocess_cf2_clean_rb_log(data2014, get_price_var_col(2014))
y=data2014_pp_rb['Class']
X=data2014_pp_rb.drop(columns=['Class'])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2,random_state=17)

best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}
model = GradientBoostingClassifier(**best_params)
model.fit(X_train, Y_train)

predict=model.predict(X_test)
print(confusion_matrix(Y_test, predict))
print(roc_auc_score(Y_test,predict))
fpr, tpr, _ = roc_curve(Y_test,predict)
roc_auc = auc(fpr,tpr)
RocCurveDisplay(fpr=fpr,tpr=tpr,roc_auc=roc_auc).plot()

## PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
growth_columns = list(filter(lambda x: "growth" in x.lower(), list(data2014.columns)))

In [ ]:
X = data2014[growth_columns]
X.fillna(X.median(),inplace=True)

In [ ]:
# Perform PCA
pca = PCA(n_components=len(growth_columns))
pca.fit(X)

# Calculate explained variance
explained_variance = pca.explained_variance_ratio_

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(explained_variance)

# Plot cumulative explained variance
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()

In [ ]:
# Create a PCA instance
pca = PCA(n_components=6)  # Specify the number of components you want to retain

# Apply PCA to the scaled data
pca_result = pca.fit_transform(X)

# Create a new dataframe to store the PCA results
df_pca_result_growth = pd.DataFrame(data=pca_result, columns=['GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6'])

# Concatenate the PCA results with the original dataframe
#df_final = pd.concat([data2014_pp_rb, df_pca_result], axis=1)

# Print the explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Print the principal components' loadings or coefficients
print("Principal Components' Loadings:")
print(pca.components_)

In [ ]:
df_pca_result_growth

In [ ]:
ratio_columns = list(filter(lambda x: "ratio" in x.lower(), list(data2014.columns)))
ratio_columns = list(filter(lambda x: "Price to Sales Ratio" not in x and "Payout Ratio" not in x and "priceToBookRatio" not in x and "priceToSalesRatio" not in x, ratio_columns))

In [ ]:
X = data2014[ratio_columns]
X.fillna(X.median(),inplace=True)

In [ ]:
# Perform PCA
pca = PCA(n_components=len(ratio_columns))
pca.fit(X)

# Calculate explained variance
explained_variance = pca.explained_variance_ratio_

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(explained_variance)

# Plot cumulative explained variance
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()

In [ ]:
# Create a PCA instance
pca = PCA(n_components=3)  # Specify the number of components you want to retain

# Apply PCA to the scaled data
pca_result = pca.fit_transform(X)

# Create a new dataframe to store the PCA results
df_pca_result_ratio = pd.DataFrame(data=pca_result, columns=['RAT1', 'RAT2', 'RAT3'])

# Concatenate the PCA results with the original dataframe
#df_final = pd.concat([data2014_pp_rb, df_pca_result], axis=1)

# Print the explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Print the principal components' loadings or coefficients
print("Principal Components' Loadings:")
print(pca.components_)

In [ ]:
df_pca_result_ratio

In [ ]:
per_share_columns = list(filter(lambda x: "per Share" in x, list(data2014.columns)))

In [ ]:
X = data2014[per_share_columns]
X.fillna(X.median(),inplace=True)

In [ ]:
# Perform PCA
pca = PCA(n_components=len(per_share_columns))
pca.fit(X)

# Calculate explained variance
explained_variance = pca.explained_variance_ratio_

# Calculate cumulative explained variance
cumulative_variance = np.cumsum(explained_variance)

# Plot cumulative explained variance
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance vs. Number of Components')
plt.grid(True)
plt.show()

In [ ]:
# Create a PCA instance
pca = PCA(n_components=5)  # Specify the number of components you want to retain

# Apply PCA to the scaled data
pca_result = pca.fit_transform(X)

# Create a new dataframe to store the PCA results
df_pca_result_per_share = pd.DataFrame(data=pca_result, columns=['PS1', 'PS2', 'PS3', 'PS4', 'PS5'])

# Concatenate the PCA results with the original dataframe
#df_final = pd.concat([data2014_pp_rb, df_pca_result], axis=1)

# Print the explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Print the principal components' loadings or coefficients
print("Principal Components' Loadings:")
print(pca.components_)

In [ ]:
df_pca_result_per_share

In [ ]:
def preprocess_cf2_clean_rb_log_new(df: pd.DataFrame,price_var_col) -> pd.DataFrame:
    clean_columns = list(filter(lambda x: "per Share" not in x and "ratio" not in x.lower() and "growth" not in x.lower(), list(df.columns)))

    df_clean = df[clean_columns]
    
    sector_dict = {}
    tmp = df_clean[['Sector', 'Class']]
    for i in tmp['Sector'].drop_duplicates():
        a=tmp[tmp['Sector']==i].value_counts().sort_index().values
        sector_dict[i] = a[1]/sum(a)
    df_copy = df_clean
    df_copy['Sector'] = df_copy['Sector'].replace(sector_dict)
    df_copy.drop(columns=['Unnamed: 0', price_var_col],axis=1,inplace=True)
    float_columns=list(df_copy.select_dtypes(include=['float64']).columns)
    float_columns.remove('Sector')


    df_for_loging = df_copy.drop(columns=['Class'])
    
    df_copy_pos = df_for_loging-df_for_loging.min()
    
    df_copy_pos_logged = np.log1p(df_copy_pos)

    df_copy_pos_logged['Class'] = df_copy['Class']
    

    scaler = RobustScaler()
    df2 = pd.concat([df_copy_pos_logged, df_pca_result_per_share, df_pca_result_growth, df_pca_result_ratio], axis=1)
    df2[float_columns] = scaler.fit_transform(df2[float_columns])
    
    lower = df2[float_columns].quantile(0.01)
    greater = df2[float_columns].quantile(0.99)
    df2[float_columns] = (df2[float_columns])[(df2[float_columns] >= lower) & (df2[float_columns] <= greater)]

    df2.fillna(df2.median(),inplace=True)

    return df2

In [ ]:
data2014_pp_rb = preprocess_cf2_clean_rb_log_new(data2014, get_price_var_col(2014))
y=data2014_pp_rb['Class']
X=data2014_pp_rb.drop(columns=['Class'])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3,random_state=17)

best_params = {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.5}
model = GradientBoostingClassifier(**best_params)
model.fit(X_train, Y_train)

predict=model.predict(X_test)
print(confusion_matrix(Y_test, predict))
print(roc_auc_score(Y_test,predict))
fpr, tpr, _ = roc_curve(Y_test,predict)
roc_auc = auc(fpr,tpr)
RocCurveDisplay(fpr=fpr,tpr=tpr,roc_auc=roc_auc).plot()

## кластеризация

In [ ]:
tsne = TSNE(random_state=17)
tsne_representation = tsne.fit_transform(X_2018_scaled)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(tsne_representation[:, 0], tsne_representation[:, 1], c=y, s=5);
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
from sklearn.manifold import TSNE

model = TSNE()

In [ ]:
transformed = model.fit_transform(final_df)

In [ ]:
x_axis = transformed[:, 0]
y_axis = transformed[:, 1]

In [ ]:
plt.scatter(x_axis, y_axis, c ="red", s =5)
plt.grid(True)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

In [ ]:
ag_silhouette= []
db_score=[]
calinski_score=[]
for i in range(1, 11):
    dbscan = DBSCAN(eps=i, min_samples=7).fit(transformed)
    preds = dbscan.labels_
    
    silhouette = silhouette_score(transformed,preds)
    ag_silhouette.append(silhouette)
    print("Silhouette score for eps {}: {}".format(i,silhouette))
    
    db = davies_bouldin_score(transformed,preds)
    db_score.append(db)
    print("Davies Bouldin score for eps {}: {}".format(i,db))

    calinski = calinski_harabasz_score(transformed,preds)
    calinski_score.append(calinski)
    print("Calinski harabasz score for eps {}: {}".format(i,calinski))
    print("-"*100)

In [ ]:
plt.figure(figsize=(7,4))
plt.title("DBSCAN model Silhouette score \nfor determining eps\n",fontsize=16)
plt.scatter(x=[i for i in range(1,11)],y=ag_silhouette,s=150,edgecolor='k')
plt.plot([i for i in range(1,11)],ag_silhouette)
plt.grid(True)
plt.grid(True)
plt.xlabel("eps",fontsize=14)
plt.ylabel("DBSCAN Silhouette score",fontsize=15)
plt.xticks([i for i in range(1,11)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("DBSCAN model Davies Bouldin score \nfor determining eps\n",fontsize=16)
plt.scatter(x=[i for i in range(1,11)],y=db_score,s=150,edgecolor='k')
plt.plot([i for i in range(1,11)],db_score)
plt.grid(True)
plt.xlabel("eps",fontsize=14)
plt.ylabel("DBSCAN Davies Bouldin score",fontsize=15)
plt.xticks([i for i in range(1,11)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("DBSCAN model Calinski harabasz score \nfor determining eps\n",fontsize=16)
plt.scatter(x=[i for i in range(1,11)],y=calinski_score,s=150,edgecolor='k')
plt.plot([i for i in range(1,11)],calinski_score)
plt.grid(True)
plt.grid(True)
plt.xlabel("eps",fontsize=14)
plt.ylabel("DBSCAN Calinski harabasz score",fontsize=15)
plt.xticks([i for i in range(1,11)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("DBSCAN model Davies Bouldin score \nfor determining eps\n",fontsize=16)
plt.scatter(x=[i for i in range(1,11)],y=db_score,s=150,edgecolor='k')
plt.plot([i for i in range(1,11)],db_score)
plt.grid(True)
plt.xlabel("eps",fontsize=14)
plt.ylabel("DBSCAN Davies Bouldin score",fontsize=15)
plt.xticks([i for i in range(1,11)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7,4))
plt.title("DBSCAN model Calinski harabasz score \nfor determining min_samples\n",fontsize=16)
plt.scatter(x=[i for i in range(1, 150, 15)],y=calinski_score,s=150,edgecolor='k')
plt.plot([i for i in range(1, 150, 15)],calinski_score)
plt.grid(True)
plt.grid(True)
plt.xlabel("Min_samples",fontsize=14)
plt.ylabel("DBSCAN Calinski harabasz score",fontsize=15)
plt.xticks([i for i in range(1, 150, 15)],fontsize=14)
plt.yticks(fontsize=15)
plt.show()

In [ ]:
preds4 = DBSCAN().fit(transformed).labels_

n_clusters_ = len(set(preds4)) - (1 if -1 in preds4 else 0)
n_noise_ = list(preds4).count(-1)
print(f'Estimated number of clusters: {n_clusters_}')
print(f'Estimated number of noise points: {n_noise_}')